In [1]:
from collections import defaultdict
import copy

### An implementation of an NFA class that computes on a input strings

In [2]:
class NFA:
    def __init__(self, transition_file):
        self.transitions = defaultdict(frozenset) 
        self.alphabet = set()
        with open(transition_file, 'r') as file:
            line = '#'
            while line.startswith('#') or line=='':
                line = next(file).strip()
            
            self.start, acc = line.split(' ')
            self.accept = acc.split(',')
            for line in file:
                line = line.strip()
                if line.startswith('#') or line=='':
                    continue
                    
                parts = line.replace(" ","").split("->")
                state, symbol = parts[0].split(",")
                if symbol:
                    self.alphabet.add(symbol)
                self.transitions[(state,symbol)] = frozenset(parts[1].split(","))


    def __repr__(self):
        ret = 'Symbols: '+(",".join(list(self.alphabet)))
        ret += "\nStart state: " + self.start
        ret += "\nAccept state(s):" + ",".join(self.accept)
        ret += "\nTransitions:"
        for state_symbol in self.transitions:
            state, symbol = state_symbol
            end_states = self.transitions[state_symbol]
            ret+=f"\n{state_symbol} -> {end_states}"
            
        return ret

    
    def epsilon_transition(self, states):
        prev_states = set()
        new_states = states.copy()
        
        while prev_states != new_states:
            prev_states = new_states.copy()
            for state in prev_states:
                new_states.update(self.transitions[(state,'')])
        
        return new_states


    def compute(self, string):
        states = self.epsilon_transition(set({self.start}))
    
        for c in string:
            next_states = set()
            for state in states:
                next_states.update(self.transitions[(state,c)])
            states = self.epsilon_transition(next_states)
                                
        for state in states:
            if state in self.accept:
                return 'accept'
        
        return 'reject'
    
    
    def star(self):
        output = copy.deepcopy(self)
        
        output.transitions[('q_start', '')] = frozenset(['q_accept', output.start])
        output.accept.append('q_accept')
        output.start = 'q_start'
        
        for state in output.accept:
            output.transitions[(state, '')] = frozenset([output.start])

        return output
    
    
    def union(self, nfa):
        output = copy.deepcopy(self)
        
        output.transitions.update(nfa.transitions)
        output.transitions[('q_start', '')] = frozenset([nfa.start, output.start])
        output.start = 'q_start'
        output.accept += nfa.accept
    
        return output
    
    
    def concat(self, nfa):
        output = copy.deepcopy(self)
        
        for state in output.accept:
            output.transitions[(state, '')] = frozenset([nfa.start]).union(output.transitions[(state, '')])
        
        output.transitions.update(nfa.transitions)
        output.accept = nfa.accept
        
        return output
        

In [3]:
#Tests
#Make sure you test your code on different nfas and check to see if the outcome is correct.
#We will not disclose our tests, you grade will be based on how many of the tests will pass

with open ('sample_transitions.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q0 q1,q2
    q0, -> q1,q2
    q1,0 -> q1
    q2,0 -> q3
    q3,1 -> q2
    """)

with open ('sample_transitions2.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q4 q5,q6
    q4, -> q5,q6
    q5,0 -> q5
    q6,0 -> q7
    q7,1 -> q6
    """)

with open ('ab_plus.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q0 q2
    q0,a -> q1
    q1,b -> q2
    q2,a -> q3
    q3,b -> q2 
    """)

with open ('b.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q4 q5
    q4,b -> q5
    """)

with open ('aa.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q4 q6
    q4,a -> q5
    q5,a -> q6
    """)

with open ('sample_transitions_1.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q1 q3,q4
    q1,0 -> q2
    q2,1 -> q3,q4
    q4, -> q9
    """)

with open ('sample_transitions_2.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q5 q7,q8
    q5,2 -> q6
    q6,1 -> q7
    q6, -> q8
    """)

nfa = NFA('./sample_transitions.txt')
nfa2 = NFA('./sample_transitions2.txt')

nfa_11 = NFA('./sample_transitions_1.txt')
nfa_22 = NFA('./sample_transitions_2.txt')

nfa_star = nfa.star()
nfa_union = nfa.union(nfa2)
nfa_concat = nfa.concat(nfa2)

nfa_1 = NFA("./ab_plus.txt")
nfa_2 = NFA("./b.txt")
nfa_test = nfa_1.union(nfa_2)

nfa_1 = NFA("./ab_plus.txt")
nfa_2 = NFA("./aa.txt")
test_nfa = nfa_1.concat(nfa_2)

assert nfa_star.compute("") == "accept", "Empty string should be accepted"
assert nfa_star.compute("0") == "accept", "0 should be accepted"
assert nfa_star.compute("01") == "accept", "01 does not belong to the language"
assert nfa_star.compute('01'*10) == "accept", "(01)^10 is part of the language."
assert nfa_star.compute('0'*10) == "accept", "0^10 is part of the language."

nfa_star = nfa_11.star()
assert nfa_star.compute("01") == "accept", "01 string should be accepted"
assert nfa_star.compute("011") == "reject", "011 string should be rejected"
assert nfa_star.compute("0101") == "accept", "0101 string should be accepted"
assert nfa_star.compute("10") == "reject", "10 string should be accepted"
assert nfa_star.compute("01"*10) == "accept", "10 string should be accepted"
assert nfa_star.compute("") == "accept", "empty string should be accepted"

assert nfa_union.compute("") == "accept", "Empty string should be accepted"
assert nfa_union.compute("0") == "accept", "0 should be accepted"
assert nfa_union.compute("010") == "reject", "010 does not belong to the language"
assert nfa_union.compute('01'*10) == "accept", "(01)^10 is part of the language."
assert nfa_union.compute('0'*10) == "accept", "0^10 is part of the language."

assert nfa_test.compute('') == 'reject', 'empty string not accepted'
assert nfa_test.compute('b') == 'accept', 'b in language'
assert nfa_test.compute('bb') == 'reject', 'bb not in language'
assert nfa_test.compute('ab') == 'accept', 'ab in language'
assert nfa_test.compute('ab' * 10) == 'accept', 'ab*10 in language'
assert nfa_test.compute('aba') == 'reject', 'aba not in language'
assert nfa_test.compute('ab' * 10 + 'a') == 'reject', 'ab * 10 + a not in language'
assert nfa_test.compute('ab' * 10 + 'b') == 'reject', "ab * 10 + b not in language"

nfa_union = nfa_11.union(nfa_22)
assert nfa_union.compute("01") == "accept", "01 string should be accepted"
assert nfa_union.compute("21") == "accept", "21 string should be accepted"
assert nfa_union.compute("20") == "reject", "20 string should be accepted"
assert nfa_union.compute("2") == "accept", "2 string should be accepted"
assert nfa_union.compute("") == "reject", "empty string should be reject"
assert nfa_union.compute("01"*10) == "reject", "(01)^10 string should be accepted"

assert nfa_concat.compute("") == "accept", "Empty string should be accepted"
assert nfa_concat.compute("0") == "accept", "0 should be accepted"
assert nfa_concat.compute("010") == "accept","010 does not belong to the language"
assert nfa_concat.compute('01'*10) == "accept", "(01)^10 is part of the language."
assert nfa_concat.compute('0'*10) == "accept", "0^10 is part of the language."

assert test_nfa.compute("") == "reject", "empty string not in language"
assert test_nfa.compute("abaa") == "accept", "abaa in language"
assert test_nfa.compute("ab" * 10 + "aa") == "accept", "ab * 10 + aa in language"
assert test_nfa.compute("ab" * 10 + "aaa") == "reject", "ab * 10 + aaa not in language"
assert test_nfa.compute("abbabaa") == "reject", "abbabaa not in language"

nfa_concat = nfa_11.concat(nfa_22)
assert nfa_concat.compute("01") == "reject", "01 string should be rejected"
assert nfa_concat.compute("0121") == "accept", "0121 string should be accepted"
assert nfa_concat.compute("012") == "accept", "012 string should be accepted"

print("ALL TESTS PASSED")

ALL TESTS PASSED


### Submissions

Please fillin the "compute" function and test your code with a number of differnt NFAs. Once you made sure it is bug free submit your jupyter notebook in Gradescope.

IMPORTANT: Please rename your notebook to your GTID.ipyn (e.g. hhassanzadeh3.ipynb) before submission.